In [76]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np

## Lets get the data, model and setup trainnig code

In [77]:
train_loader = DataLoader(datasets.MNIST("./data", train=True, transform=transforms.ToTensor(), download=True), batch_size=128, shuffle=True)
test_loader = DataLoader(datasets.MNIST("./data", train=False, transform=transforms.ToTensor(), download=True), batch_size=128, shuffle=False)

print(f"Training images {len(train_loader.dataset)}, Test images {len(test_loader.dataset)}")

Training images 60000, Test images 10000


In [63]:
class mnist_model(nn.Module):
  def __init__(self):
    super(mnist_model, self).__init__()
    self.layer1 = nn.Conv2d(1, 20, kernel_size=5, stride=1, padding=0)
    self.layer2 = nn.Conv2d(20, 50, kernel_size=5, stride=1, padding=0)
    self.layer3 = nn.Linear(800, 500, bias=True)
    self.layer4 = nn.Linear(500, 10, bias=True)

    self.act = nn.ReLU()
    self.pool = nn.MaxPool2d((2, 2))

  def forward(self, x):
    out = self.act(self.layer1(x))
    out = self.pool(out)
    out = self.act(self.layer2(out))
    out = self.pool(out)
    out = out.view(-1, 800)
    out = self.act(self.layer3(out))
    # out = self.act(self.layer4(out))
    out = self.layer4(out)
    return out

  def output(self, x):
    out1 = self.act(self.layer1(x))
    out1 = self.pool(out1)
    out2 = self.act(self.layer2(out1))
    out2 = self.pool(out2)
    out2 = out2.view(-1, 800)
    out3 = self.act(self.layer3(out2))
    out4 = self.act(self.layer4(out3))    
    return out1, out2, out3, out4

In [64]:
model = mnist_model()
print(model)

epochs = 0
lr = 0.1

USE_MSE = False
optimizer = optim.SGD(model.parameters(), lr=lr)
criterion = nn.MSELoss() if USE_MSE else nn.CrossEntropyLoss()
# criterion = nn.MSELoss()
# lrs = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)

mnist_model(
  (layer1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (layer2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (layer3): Linear(in_features=800, out_features=500, bias=True)
  (layer4): Linear(in_features=500, out_features=10, bias=True)
  (act): ReLU()
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
)


In [65]:
import os
# GPU run
gpu_list = [0, 1, 2, 3]
gpu_list_str = ','.join(map(str, gpu_list))
os.environ.setdefault("CUDA_VISIBLE_DEVICES", gpu_list_str)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.nn.DataParallel(model)
model.to(device)

mnist_model(
  (layer1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (layer2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (layer3): Linear(in_features=800, out_features=500, bias=True)
  (layer4): Linear(in_features=500, out_features=10, bias=True)
  (act): ReLU()
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
)



```
# This is formatted as code
```

## Training

In [66]:
def get_acc(model, loader):
  correct = 0
  total = 0
  for img, label in loader:
    correct += torch.sum(torch.argmax(model(img.to(device)), -1) == label.to(device)).item()
    total += len(img)
  return 100*correct/total

In [67]:
for e in range(epochs):
  print("lr", optimizer.param_groups[0]["lr"])
  for img, label in train_loader:
    # print(img.shape, label.shape)
    label_onehot = nn.functional.one_hot(label.to(device), num_classes=10).float()
    out = model(img.to(device))
    optimizer.zero_grad()
    loss = criterion(out, label_onehot)
    loss.backward()
    optimizer.step()
  # lrs.step()
  print(f"Epoch {e}, training accuracy {get_acc(model, train_loader)}, test accuracy {get_acc(model, test_loader)}")

## Extract weights

In [68]:
params = [(name, p.data.cpu().numpy()) for (name, p) in model.named_parameters()]

In [69]:
for (name, p) in params:
  print(f"Layer {name.split('.')[0]}, type {name.split('.')[1]}, shape {p.shape}")

Layer layer1, type weight, shape (20, 1, 5, 5)
Layer layer1, type bias, shape (20,)
Layer layer2, type weight, shape (50, 20, 5, 5)
Layer layer2, type bias, shape (50,)
Layer layer3, type weight, shape (500, 800)
Layer layer3, type bias, shape (500,)
Layer layer4, type weight, shape (10, 500)
Layer layer4, type bias, shape (10,)


In [70]:
#print(params)

## Visualize hidden activations

In [71]:
# print(model.children())
# out = list(model.children())[0](img.cuda()).data.cpu().numpy()

In [72]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# for _ in range(out.shape[1]):
#   plt.figure(figsize=(1, 1))
#   plt.imshow(out[0, 0], cmap="gray")

In [73]:
for img, label in train_loader:
  break

In [74]:
# from google.colab import drive
# drive.mount('/content/drive')
# path = "/content/drive/My Drive/Colab Notebooks/Falcon Neural Network/LeNet/"

In [75]:
import os
subdir = 'trained' if epochs != 0 else 'init'
path = f"{os.path.expanduser('~')}/output/{subdir}/LeNet/"
if not os.path.exists(path):
    os.makedirs(path)

print(f'Save parameters to {path}')

np.savetxt(fname=path+"input_0", delimiter=" ", X=img.cuda().view(-1, 784).tolist())
np.savetxt(fname=path+"outputlayer1_0", delimiter=" ", X=model.output(img.cuda().view(128, 1, 28, 28))[0].data.cpu().view(-1))
np.savetxt(fname=path+"outputlayer2_0", delimiter=" ", X=model.output(img.cuda().view(128, 1, 28, 28))[1].tolist())
np.savetxt(fname=path+"outputlayer3_0", delimiter=" ", X=model.output(img.cuda().view(128, 1, 28, 28))[2].tolist())
np.savetxt(fname=path+"outputlayer4_0", delimiter=" ", X=model.output(img.cuda().view(128, 1, 28, 28))[3].tolist())

np.savetxt(fname=path+"weight1_0", delimiter=" ", X=params[0][1].reshape(5*5*1, 20).tolist())
np.savetxt(fname=path+"bias1_0", delimiter=" ", X=params[1][1].tolist())
np.savetxt(fname=path+"weight2_0", delimiter=" ", X=params[2][1].reshape(5*5*20, 50).tolist())
np.savetxt(fname=path+"bias2_0", delimiter=" ", X=params[3][1].tolist())
np.savetxt(fname=path+"weight3_0", delimiter=" ", X=params[4][1].tolist())
np.savetxt(fname=path+"bias3_0", delimiter=" ", X=params[5][1].tolist())
np.savetxt(fname=path+"weight4_0", delimiter=" ", X=params[6][1].tolist())
np.savetxt(fname=path+"bias4_0", delimiter=" ", X=params[7][1].tolist())

Save parameters to /home/haoqi.whq/output/init/LeNet/
